In [3]:
from google.colab import userdata

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Config

In [5]:
BASE_DIR="/content/drive/MyDrive/Databricks_genai_hackathon_jan2024"
DATA_FILE_PATH=f"{BASE_DIR}/datasets/WMT_Grocery_202209.csv"
VECTORDB_PATH=f"{BASE_DIR}/chromadb1"
EMBEDDING_MODEL_PATH=f"{BASE_DIR}/embedding_model"

# Installs

In [6]:
!pip3 install --upgrade --quiet langchain langchain-community langchain-openai transformers chromadb tiktoken sentence-transformers langsmith langchainhub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.7/806.7 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 76.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 36.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 237.0/237.0 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

# Imports

In [7]:
import pandas
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain_core.tracers.context import tracing_v2_enabled
from langchain_community.vectorstores import Chroma
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import DataFrameLoader
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, PromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, get_buffer_string
from langchain.schema import format_document
from operator import itemgetter
from langchain.memory import ConversationBufferMemory

# Vector DB

In [8]:
retriever = Chroma(persist_directory=VECTORDB_PATH,
                   embedding_function=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_PATH)).as_retriever(search_kwargs={"k": 10})

In [9]:
[_.page_content for _ in retriever.get_relevant_documents("Hummus")]

['Marketside Classic Hummus, 10 Oz',
 'Marketside Pine Nut Hummus, 10 oz',
 'Marketside Roasted Red Pepper Hummus, 10 Oz',
 'Marketside Everything Hummus, 10 oz',
 'Marketside Spicy Hummus, 10 oz',
 'Marketside Roasted Garlic Hummus, 10 Oz',
 'Fresh Cravings Classic Hummus 10oz',
 'Fresh Cravings Roasted Red Pepper Hummus 10oz',
 'Fresh Cravings Roasted Garlic Hummus 10 oz',
 'Fresh Cravings Everything Bagel Hummus 10oz']

# Simple Order Bot

We need to chains:
1. Chat history chain
2. Order chain

**Chat history chain** is responsible for providing the chat history throughout the order process. It acts a message collector.

**Order chain** is responsible for talking to the user in order to complete an order.



In [85]:
# Setting up langsmith
import os
from langsmith import Client

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = userdata.get('LANGCHAIN_TRACING_V2')
os.environ["LANGCHAIN_ENDPOINT"] = userdata.get('LANGCHAIN_ENDPOINT')
os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGCHAIN_PROJECT')

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature=0,
                 openai_api_key=userdata.get('OPENAI_API_KEY')
                 )

retriever = Chroma(persist_directory=VECTORDB_PATH,
                   embedding_function=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_PATH)).as_retriever(search_kwargs={"k": 10})


prompt = ChatPromptTemplate.from_messages(
    [("system", "You are OrderBot, an automated service to collect orders for a convenience grocery store. You first greet the customer, then collect the order, and then ask if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Make sure to clarify all options, extras and sizes to uniquely identify the item from the product catelog. You respond in a short, very conversational friendly style. The products include: Hummus 1, Hummus 2, Hummus 3, Coke 1, Coke 2, Coke 3"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})
{'history': []}

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | llm
    # | StrOutputParser()
)

In [86]:
inputs = {"input": "Hi"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response.content})
response

AIMessage(content='Hello! Welcome to our convenience grocery store. What can I get for you today?')

In [87]:
memory.load_memory_variables({})
inputs = {"input": "I want Hummus"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response.content})
response

AIMessage(content='Sure! Which type of hummus would you like? We have Hummus 1, Hummus 2, and Hummus 3.')

In [88]:
memory.load_memory_variables({})
inputs = {"input": "Hummus 1, please"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response.content})
response

AIMessage(content="Great choice! Is there anything else you'd like to add to your order, or is that all for now?")

In [89]:
memory.load_memory_variables({})
inputs = {"input": "Do you have pizza as well?"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response.content})
response

AIMessage(content="I'm sorry, but we currently don't have pizza in our store. Is there anything else you'd like to add to your order?")

In [90]:
memory.load_memory_variables({})
inputs = {"input": "No, that's it."}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response.content})
response

AIMessage(content='Got it. Is this order for pickup or delivery?')

In [91]:
memory.load_memory_variables({})
inputs = {"input": "Pickup"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response.content})
response

AIMessage(content="Perfect! Your order is Hummus 1 for pickup. Is there anything else you'd like to add before we finalize your order?")

In [92]:
memory.load_memory_variables({})
inputs = {"input": "No"}
response = chain.invoke(inputs)
memory.save_context(inputs, {"output": response.content})
response

AIMessage(content='Great! Your order for pickup is Hummus 1. Thank you for shopping with us!')

In [94]:
response.content

'Great! Your order for pickup is Hummus 1. Thank you for shopping with us!'



---



# Version 2

In [116]:

# Setting up langsmith
import os
from langsmith import Client

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = userdata.get('LANGCHAIN_TRACING_V2')
os.environ["LANGCHAIN_ENDPOINT"] = userdata.get('LANGCHAIN_ENDPOINT')
os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGCHAIN_PROJECT')

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature=0,
                 openai_api_key=userdata.get('OPENAI_API_KEY')
                 )

retriever = Chroma(persist_directory=VECTORDB_PATH,
                   embedding_function=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_PATH)).as_retriever(search_kwargs={"k": 10})


prompt = ChatPromptTemplate.from_messages(
    [("system", "You are OrderBot, an automated service to collect orders for a convenience grocery store. You first greet the customer, then collect the order, and then ask if it's a pickup or delivery. You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. If it's a delivery, you ask for an address. Make sure to clarify all options, extras and sizes to uniquely identify the item from the product catelog. You respond in a short, very conversational friendly style. The products include: Hummus 1, Hummus 2, Hummus 3, Coke 1, Coke 2, Coke 3. Once the order finalized, include 'Bye!' in your final prompt."),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})
{'history': []}

chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | llm
    # | StrOutputParser()
)

In [98]:
in_progress = True

while in_progress:
    memory.load_memory_variables({})
    user_input = input("User: ")
    inputs = {"input": user_input}
    response = chain.invoke(inputs)
    memory.save_context(inputs, {"output": f"Bot: {response.content}"})
    print(f"{response.content}")
    if "Bye!" in response.content:
        in_progress = False

User: Hi
Hello! Welcome to our convenience grocery store. What can I get for you today?
User: Do you have hummus?
Bot: Yes, we have three types of hummus available: Hummus 1, Hummus 2, and Hummus 3. Which one would you like to order?
User: hummus 1 please
Bot: Great choice! Is there anything else you'd like to add to your order, or is that all for now?
User: Do you have pizza?
Bot: I'm sorry, but we don't have pizza. Is there anything else you'd like to add to your order, or are you ready to finalize it?
User: No
Bot: Bot: Got it! Is this order for pickup or delivery?
User: pickup
Bot: Bot: Perfect! Your order is Hummus 1 for pickup. Is there anything else you'd like to add before I finalize it?
User: no
Bot: Bot: Bot: Alright, your order for Hummus 1 for pickup is all set. Thank you for shopping with us! Bye!




---



# Get list of products from vector DB

In [17]:
# Setting up langsmith
import os
from langsmith import Client

os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')
os.environ["LANGCHAIN_TRACING_V2"] = userdata.get('LANGCHAIN_TRACING_V2')
os.environ["LANGCHAIN_ENDPOINT"] = userdata.get('LANGCHAIN_ENDPOINT')
os.environ["LANGCHAIN_PROJECT"] = userdata.get('LANGCHAIN_PROJECT')

llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125",
                 temperature=0,
                 openai_api_key=userdata.get('OPENAI_API_KEY')
                 )

retriever = Chroma(persist_directory=VECTORDB_PATH,
                   embedding_function=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL_PATH)).as_retriever(search_kwargs={"k": 10})

system_template = """
You are OrderBot, an automated service to collect orders for a convenience grocery store. \
You first greet the customer, then collect the order, and then ask if it's a pickup or delivery. \
You wait to collect the entire order, then summarize it and check for a final time if the customer wants to add anything else. \
If it's a delivery, you ask for an address. \
Make sure to clarify all options, extras and sizes to uniquely identify the item from the product catelog. \
You respond in a short, very conversational friendly style. \

The products is included below:

{product_catalog}

Once the order finalized, include 'Bye!' in your final prompt."""

user_template = "User: {input}"

system_prompt = SystemMessagePromptTemplate.from_template(system_template)
user_prompt = HumanMessagePromptTemplate.from_template(user_template)

prompt = ChatPromptTemplate.from_messages([
    system_prompt,
    MessagesPlaceholder(variable_name="history"),
    user_prompt,
])

memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

loaded_memory = RunnablePassthrough.assign(history=RunnableLambda(memory.load_memory_variables) | itemgetter("history"))

chain = (loaded_memory |
    {
        "input": lambda x: x["input"],
        "product_catalog": itemgetter("input") | retriever,
        "history": lambda x: x["history"],
    }
    | prompt
    | llm
)

In [46]:
in_progress = True

while in_progress:
    memory.load_memory_variables({})
    user_input = input("User: ")
    inputs = {"input": user_input}
    response = chain.invoke(inputs)
    memory.save_context(inputs, {"output": f"{response.content}"})
    print(f"History: {memory.load_memory_variables({})}")
    print(f"{response.content}")
    if "Bye!" in response.content:
        in_progress = False

User: hi
OrderBot: Hello! How can I assist you today?
User: do you have milk?
OrderBot: I'm sorry, but we don't have milk in our product catalog. Is there anything else you would like to order?
User: what about hummus?
OrderBot: Yes, we have a variety of hummus options available. Here are the flavors we offer:
1. Marketside Pine Nut Hummus, 10 oz
2. Marketside Classic Hummus, 10 oz
3. Marketside Everything Hummus, 10 oz
4. Marketside Roasted Red Pepper Hummus, 10 oz
5. Marketside Spicy Hummus, 10 oz
6. Marketside Roasted Garlic Hummus, 10 oz
7. Fresh Cravings Classic Hummus 10 oz
8. Fresh Cravings Roasted Red Pepper Hummus 10 oz
9. Fresh Cravings Roasted Garlic Hummus 10 oz
10. Fresh Cravings Everything Bagel Hummus 10 oz

What flavor would you like to order?
User: number 1 please
OrderBot: Great choice! Adding Marketside Pine Nut Hummus, 10 oz to your order. Is there anything else you would like to add, or is there anything specific you're looking for today?
User: do you have somethin